# SNS injection painting

In [ ]:
import sys
import importlib
import os
from os.path import join

import numpy as np
import pandas as pd
from matplotlib import animation
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from tqdm import tqdm
from tqdm import trange
import proplot as pplt 
import seaborn as sns

sys.path.append('..')
from tools import animation as myanim
from tools import beam_analysis as ba
from tools import plotting as myplt
from tools import utils

## Settings

In [ ]:
plt.rcParams['animation.html'] = 'jshtml'
plt.rcParams['savefig.dpi'] = 'figure'
plt.rcParams['savefig.transparent'] = False
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['grid.alpha'] = 0.04
plt.rcParams['axes.grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'

lineplot_kws = dict(marker=None, legend=False)
savefig_kws = dict(facecolor='white', dpi=300)

n_bins_hist2d = 100

In [ ]:
folder = '_output/data/'
location = 'injection point' # {'injection point', 'rtbt entrance'}

In [ ]:
utils.delete_files_not_folders('_output/figures/')

## Injection region closed orbit

In [ ]:
inj_region_coords_t0 = np.load(join(folder, 'inj_region_coords_t0.npy'))
inj_region_coords_t1 = np.load(join(folder, 'inj_region_coords_t1.npy'))
inj_region_positions_t0 = np.load(join(folder, 'inj_region_positions_t0.npy'))
inj_region_positions_t1 = np.load(join(folder, 'inj_region_positions_t1.npy'))
inj_region_coords_t0 *= 1000. # convert to mm-mrad
inj_region_coords_t1 *= 1000. # convert to mm-mrad

In [ ]:
fig, ax = pplt.subplots(figsize=(6, 2.5))
ax.plot(inj_region_positions_t0, inj_region_coords_t0[:, 0], label='x (initial)')
ax.plot(inj_region_positions_t0, inj_region_coords_t0[:, 2], label='y (initial)')
ax.format(cycle='colorblind')
ax.plot(inj_region_positions_t1, inj_region_coords_t1[:, 0], ls='--', lw=1, label='x (final)')
ax.plot(inj_region_positions_t1, inj_region_coords_t1[:, 2], ls='--', lw=1, label='y (final)')
ax.format(title='Injection region closed orbit')
ax.legend(ncols=1, loc=(1.02, 0), handlelength=1.5);
ax.format(xlabel='s [m]', ylabel='[mm]') 
ax.grid(axis='y')
plt.savefig('_output/figures/inj_region_closed_orbit.png', **savefig_kws)

## Kicker strengths

In [ ]:
# def get_perveance(kin_energy, mass, line_density):
#     classical_proton_radius = 1.53469e-18 # m
#     gamma = 1 + (kin_energy / mass) # Lorentz factor
#     beta = np.sqrt(1 - (1 / gamma)**2) # velocity/speed_of_light
#     return (2 * classical_proton_radius * line_density) / (beta**2 * gamma**3)

# ring_length = 248.0
# zlim = (135.0 / 360.0) * ring_length
# zmin, zmax = -zlim, zlim
# bunch_length = zmax - zmin

# # Production beam
# kin_energy = 1.0 # [GeV]
# mass = 0.93827231 # [kg]
# intensity = 1.5e14
# Q = get_perveance(kin_energy, mass, intensity / bunch_length)
# xmax = ymax = 26.0
# area = xmax * ymax
# print('Q = {}'.format(Q))
# density_production = Q / area
# print('Q / area = {}'.format(density_production))

# # SCBD
# kin_energy = 1.0 # [GeV]
# mass = 0.93827231 # [kg]
# max_intensity = 1.5e14
# max_n_turns = 1000.
# xmax = ymax = 26.0
# area = xmax * ymax
# densities = []
# turns = np.linspace(1, max_n_turns, 1000)
# for t in turns:
#     tau = t / max_n_turns
#     intensity = max_intensity * tau
#     Q = get_perveance(kin_energy, mass, intensity / bunch_length)
#     densities.append(Q / area)
# densities = np.array(densities)
    
# fig, ax = pplt.subplots()
# ax.plot(turns, densities, color='black');
# ymin, ymax = ax.get_ylim()
# alpha = 0.15
# ax.fill_between(turns, 0., density_production, color='green', alpha=alpha)
# ax.fill_between(turns, density_production, ymax, color='red', alpha=alpha)
# ax.format(grid=True, ylim=(0, ax.get_ylim()[1]),
#           xlabel='# turns to reach full beam size', ylabel=r'Q / area [mm$^{-2}$]')
# plt.savefig('_output/figures/perveance_scaling_{}_{}.png'.format(kin_energy, xmax))

In [ ]:
kicker_angles_t0 = np.loadtxt(folder + 'kicker_angles_t0.dat')
kicker_angles_t1 = np.loadtxt(folder + 'kicker_angles_t1.dat')
kicker_names = ['ikickh_a10', 'ikickv_a10', 'ikickh_a11', 'ikickv_a11',
                'ikickv_a12', 'ikickh_a12', 'ikickv_a13', 'ikickh_a13']

In [ ]:
def waveform(t, k0, k1):
    return k0 - (k0 - k1)*np.sqrt(t)

In [ ]:
t = np.linspace(0, 1, 1000)

fig, axes = pplt.subplots(nrows=4, ncols=2, figsize=(3.5, 6))
for k0, k1, name, ax in zip(kicker_angles_t0, kicker_angles_t1, kicker_names, axes):
    ax.plot(t, 1000 * waveform(t, k0, k1), c='k')
    ax.format(title=name)
axes.format(ylabel='Amplitude', suptitle='Kicker angles', xlabel='time [ms]')
plt.savefig('_output/figures/kicker_angles.png', **savefig_kws)

In [ ]:
fig, axes = pplt.subplots(nrows=4, ncols=2, figsize=(3.5, 6))
for k0, k1, name, ax in zip(kicker_angles_t0, kicker_angles_t1, kicker_names, axes):
    ax.plot(t, waveform(t, 1.0, k1/k0), c='k')
    ax.format(title=name)
axes.format(ylabel='Amplitude', suptitle='Kicker waveforms', xlabel='time [ms]')
plt.savefig('_output/figures/kicker_waveforms.png', **savefig_kws)

## Beam statistics

In [ ]:
suffix = 'rtbt'

In [ ]:
turns = np.loadtxt(join(folder, 'turns_stored_{}.dat'.format(suffix)))
print(turns)

In [ ]:
coords = utils.load_stacked_arrays(join(folder, 'coords_{}.npz'.format(suffix)))
for i in trange(len(coords)):
    coords[i][:, 5] *= 1000. # convert dE to [MeV]

In [ ]:
moments_list = []
for X in tqdm(coords):
    Sigma = np.cov(X[:, :4].T)
    moments_list.append(ba.mat2vec(Sigma))
moments_list = np.array(moments_list)
    
stats = ba.StatsReader()
stats.read_moments(moments_list)

In [ ]:
fig, ax = pplt.subplots(figsize=(4, 3.0))
t = 500
g1 = ax.plot(turns, stats.twiss2D.loc[:t, 'eps_x'], **lineplot_kws)
g2 = ax.plot(turns, stats.twiss2D.loc[:t, 'eps_y'], **lineplot_kws)
g3 = ax.plot(turns, stats.twiss4D.loc[:t, 'eps_1'], **lineplot_kws)
g4 = ax.plot(turns, stats.twiss4D.loc[:t, 'eps_2'], **lineplot_kws)
ax.legend([g1, g2, g3, g4], labels=[r'$\varepsilon_{}$'.format(v) for v in ['x', 'y', '1', '2']], 
          ncols=1, loc='upper left', framealpha=1.0)
ax.format(ylabel='[mm mrad]', xlabel='Turn number', ygrid=True, ylim=(0., ax.get_ylim()[1]));
plt.savefig('_output/figures/emittances.png', **savefig_kws)

In [ ]:
exey = (stats.twiss2D['eps_x'] * stats.twiss2D['eps_y']).values
e1e2 = (stats.twiss4D['eps_1'] * stats.twiss4D['eps_2']).values

fig, ax = pplt.subplots(figsize=(4.0, 2.5))
g1 = ax.plot(turns, e1e2, color='red', **lineplot_kws)
g2 = ax.plot(turns, exey, color='blue', **lineplot_kws)
ax.legend([g1, g2], labels=[r'$\varepsilon_1\varepsilon_2$', r'$\varepsilon_x\varepsilon_y$'],
          ncols=1, loc='r')
ax.format(xlabel='Turn number', ylabel=r'[mm$^2$ mrad$^2$]', 
          ygrid=True, ylim=(0., ax.get_ylim()[1]))
plt.savefig('_output/figures/emittances_4D.png', **savefig_kws)

In [ ]:
fig, axes = pplt.subplots(nrows=2, figsize=(4.0, 4.0), spany=False)
g1 = axes[0].plot(turns, stats.twiss2D['eps_x'], **lineplot_kws)
g2 = axes[0].plot(turns, stats.twiss2D['eps_y'], **lineplot_kws)
g3 = axes[0].plot(turns, stats.twiss4D['eps_1'], **lineplot_kws)
g4 = axes[0].plot(turns, stats.twiss4D['eps_2'], **lineplot_kws)
axes[0].legend(handles=[g1, g2, g3, g4],
               labels=[r'$\varepsilon_{}$'.format(v) for v in ['x', 'y', '1', '2']],
               ncols=1, loc='r')
g1 = axes[1].plot(turns, e1e2, color='red', **lineplot_kws)
g2 = axes[1].plot(turns, exey, color='blue8', **lineplot_kws)
axes[1].legend(handles=[g1, g2],
               labels=[r'$\varepsilon_1\varepsilon_2$', r'$\varepsilon_x\varepsilon_y$'],
               ncols=1, loc='r')
axes[0].format(ylabel='[mm mrad]', xlabel='Turn number', ylim=(0., axes[0].get_ylim()[1]))
axes[1].format(ylabel=r'[mm$^2$ mrad$^2$]', ylim=(-0., axes[1].get_ylim()[1]))
for ax in axes:
    ax.grid(axis='y')
plt.savefig('_output/figures/emittances_combined.png', **savefig_kws)

In [ ]:
fig, ax = pplt.subplots(figsize=(3.5, 2.5))
C = 1.0 - (e1e2) / (exey)
ax.plot(turns, C, c='k', **lineplot_kws)
ax.format(xlabel='Turn number', 
          title=r'C = 1 - $\frac{\varepsilon_1\varepsilon_2}{\varepsilon_x\varepsilon_y}$', 
          ygrid=True)
plt.savefig('_output/figures/coupling_factor.png', **savefig_kws)

In [ ]:
fig, ax = pplt.subplots(figsize=(4.0, 2.25))
h1 = ax.plot(turns, stats.twiss2D['beta_x'])
h2 = ax.plot(turns, stats.twiss2D['beta_y'])
legend_kws = dict(loc='right', ncol=1, framealpha=0.)
ax.legend(handles=[h1, h2], labels=[r'$\beta_x$', r'$\beta_y$'], **legend_kws)
ax.format(xlabel='Turn number', ygrid=True, ylabel='[m/rad]')
plt.savefig('_output/figures/beta.png', **savefig_kws)

In [ ]:
fig, ax = pplt.subplots(figsize=(4.0, 2.25))
h1 = ax.plot(turns, stats.twiss2D['alpha_x'])
h2 = ax.plot(turns, stats.twiss2D['alpha_y'])
legend_kws = dict(loc='right', ncol=1, framealpha=0.)
ax.legend(handles=[h1, h2], labels=[r'$\alpha_x$', r'$\alpha_y$'], **legend_kws)
ax.format(xlabel='Turn number', ygrid=True, ylabel='[rad]')
plt.savefig('_output/figures/alpha.png', **savefig_kws)

In [ ]:
fig, axes = pplt.subplots(nrows=3, figsize=(3.5, 5.0), spany=False, aligny=True)
h1 = axes[0].plot(turns, stats.twiss2D['beta_x'], **lineplot_kws)
h2 = axes[0].plot(turns, stats.twiss2D['beta_y'], **lineplot_kws)
axes[0].format(ylabel='[m/rad]')
axes[1].format(ylabel='[rad]')
axes[2].format(ylabel='[mm mrad]')
axes[1].plot(turns, stats.twiss2D['alpha_x'], **lineplot_kws)
axes[1].plot(turns, stats.twiss2D['alpha_y'], **lineplot_kws)
axes[2].plot(turns, stats.twiss2D['eps_x'], **lineplot_kws)
axes[2].plot(turns, stats.twiss2D['eps_y'], **lineplot_kws)
legend_kws = dict(loc='r', ncol=1, framealpha=0.)
axes[0].legend(handles=[h1, h2], labels=[r'$\beta_x$', r'$\beta_y$'], **legend_kws)
axes[1].legend(handles=[h1, h2], labels=[r'$\alpha_x$', r'$\alpha_y$'], **legend_kws)
axes[2].legend(handles=[h1, h2], labels=[r'$\varepsilon_x$', r'$\varepsilon_y$'], **legend_kws)
axes.format(xlabel='Turn number', ygrid=True)
plt.savefig('_output/figures/twiss2D.png', **savefig_kws)

In [ ]:
fig, axes = pplt.subplots(nrows=2, figsize=(3.5, 3.33), spany=False, aligny=True)
axes[0].plot(turns, stats.twiss4D['u'], color='black', **lineplot_kws)
axes[1].plot(turns, stats.twiss4D['nu'], color='black', **lineplot_kws)
axes[0].format(ylabel='u')
axes[1].format(ylabel=r'$\nu$', yformatter='deg')
axes.format(ygrid=True)
plt.savefig('_output/figures/u_and_nu.png', **savefig_kws)

In [ ]:
stats.twiss2D.head()

In [ ]:
stats.twiss4D.head()

## Tunes 

In [ ]:
# def load_pybunch(filename):
#     X = pd.read_table(filename, sep=' ', skiprows=15, index_col=False, 
#                         names=['x','xp','y','yp','z','dE', 'mux', 'muy', 'nux', 'nuy', 'Jx', 'Jy'])
#     X.iloc[:, :4] *= 1000. # convert from m-rad to mm-mrad
#     X.iloc[:, 5] *= 1000. # convert energy spread from [GeV] to [MeV]
#     return X

# filenames = [
#     'bunch_turn=100.dat',
#     'bunch_turn=200.dat',
#     'bunch_turn=300.dat', 
# #     'bunch_turn=400.dat'
# ]
# for filename in filenames:
#     df = load_pybunch(os.path.join(folder, filename))
    
#     xmin, xmax = 0.0, 0.18
#     limits = [(xmin, xmax), (xmin, xmax)]
    
#     fig, ax = pplt.subplots(figsize=(2.75, 2.75))
#     nux = df['nux'].values
#     nuy = df['nuy'].values
#     nux[np.where(nux > 0.5)] -= 1.0
#     nuy[np.where(nuy > 0.5)] -= 1.0
#     nux += 6.0
#     nuy += 6.0
#     nu_limits = np.array([-0.05, 0.20])
#     nu_limits += 6.0
#     ax.hist2d(nux, nuy, bins=70, range=(nu_limits, nu_limits), cmap='dusk_r', discrete=False, colorbar=False)
#     line_kws = dict(color='white', lw=0.3, alpha=0.5)
#     ax.axvline(6.0, **line_kws)
#     ax.axhline(6.0, **line_kws)
#     ax.plot(ax.get_xlim(), ax.get_xlim(), **line_kws)
#     ax.format(xlabel=r'$\nu_x$', ylabel=r'$\nu_y$', aspect=1.0)
    
#     plt.savefig('_output/figures/tunes_{}.png'.format(filename), **savefig_kws)
#     plt.show()

In [ ]:
# fig, axes = pplt.subplots(nrows=2, figsize=(3, 4.5), height_ratios=[0.5, 1.0], sharex=False, sharey=False)

# g1 = axes[0].plot(stats.twiss2D['eps_x'])
# g2 = axes[0].plot(stats.twiss2D['eps_y'])
# g3 = axes[0].plot(stats.twiss4D['eps_1'])
# g4 = axes[0].plot(stats.twiss4D['eps_2'])
# axes[0].format(ylabel='')
# # ax.legend([g1, g2, g3, g4], labels=[r'$\varepsilon_{}$'.format(v) for v in ['x', 'y', '1', '2']], 
# #           ncols=1, loc='r', framealpha=0.)
# # ax.format(ylabel='[mm mrad]', xlabel='Turn number', ygrid=True, 
# #           ylim=(0., ax.get_ylim()[1])
# # #           ylim=(0., 20.),
# #          );
# # plt.savefig('_output/figures/emittances.png', **savefig_kws)

# axes[1].hist2d(nux, nuy, bins=70, range=((0.0, 0.25), (0.0, 0.25)), cmap='dusk_r', discrete=False)
# line_kws = dict(color='white', lw=0.5)
# axes[1].axvline(0.18, **line_kws)
# axes[1].axhline(0.18, **line_kws)
# axes[1].format(xlabel=r'$\nu_x$', ylabel=r'$\nu_y$', aspect=1.0)
# # plt.savefig('_output/figures/tunes.png', **savefig_kws)
# plt.show()

## TBT coordinates 

In [ ]:
foil_pos = (48.6, 46.0)
coords_foil_frame = []
for X in tqdm(coords):
    Y = np.copy(X)
    Y[:, 0] -= foil_pos[0]
    Y[:, 2] -= foil_pos[1]
    coords_foil_frame.append(Y)

In [ ]:
turn = -1
plot_kws = dict(bins=125, sigma=3.0, hist_height_frac=0.6, cmap='mono', blur=0.5)

In [ ]:
axes = myplt.corner(coords_foil_frame[turn][:, :4], **plot_kws)
limits = [ax.get_xlim() for ax in axes[-1, :]]
plt.savefig('_output/figures/corner4D_turn{}.png'.format(turn), **savefig_kws)

In [ ]:
# from skimage import filters

# i, j = 0, 3
# bins = 125

# labels = ["x [mm]", "x' [mrad]", "y [mm]", "y' [mrad]"]

# for cmap in ['mono', 'dusk', 'dusk_r', 'mono_r']:

#     heights_list = []
#     vmax = 0
#     for turn in [100, 499]:
#         X = coords_foil_frame[turn]
#         heights, xedges, yedges = np.histogram2d(X[:, i], X[:, j], bins, (limits[i], limits[j]))
#         heights = filters.gaussian(heights, 0.5)
#         heights_list.append(heights)
#         vmax = max(vmax, heights.max())
#     vmax = None
#     fig, axes = pplt.subplots(ncols=2, figsize=None)
#     axes[0].pcolormesh(xedges, yedges, heights_list[0].T, cmap=cmap, vmax=vmax)
#     axes[1].pcolormesh(xedges, yedges, heights_list[1].T, cmap=cmap, colorbar=False, colorbar_kw=dict(width=0.1, space=1.0), vmax=vmax)
# #         fig, ax = pplt.subplots(figsize=None)
# #         ax.pcolormesh(xedges, yedges, heights.T, cmap=cmap,)
# # #         ax.imshow(np.flip(heights, axis=0), cmap=cmap, interpolation='gaussian')
#     fontsize = 'large'
#     myplt.despine(axes)
#     axes.format(xlabel=labels[i], ylabel=labels[j], 
#                 xlabel_kw=dict(fontsize=fontsize), ylabel_kw=dict(fontsize=fontsize)
#                )
#     plt.savefig('/Users/46h/Desktop/xyp_{}.png'.format(cmap), **savefig_kws)
# #         plt.show()

In [ ]:
axes = myplt.corner(coords_foil_frame[turn], **plot_kws)
plt.close()
limits = [ax.get_xlim() for ax in axes[-1, :]]
limits[4] = (-248/2, 248/2)
axes = myplt.corner(coords_foil_frame[turn], limits=limits, **plot_kws)
plt.savefig('_output/figures/corner6D_turn{}.png'.format(turn), **savefig_kws)

In [ ]:
anim_kws = dict(skip=19, keep_last=True, text_fmt='Turn = {}', limits=limits)
for key in ['bins', 'hist_height_frac', 'cmap', 'blur']:
    anim_kws[key] = plot_kws[key]

In [ ]:
# fig, axes = pplt.subplots(ncols=3, nrows=3, figsize=(5, 5), spanx=False, spany=False)

# for j in range(3):
#     for i in range(j):
#         axes[i, j].axis('off')
# myplt.despine(axes, ('top', 'right'))
# plot_kws = dict(lw=0, marker='.', ms=1, color='steelblue', markeredgecolor='none')
# lines = [[], [], []]
# for i in range(3):
#     for j in range(i + 1):
#         line, = axes[i, j].plot([], [], **plot_kws)
#         lines[i].append(line)
        
#         axes[i, j].format(xlim=limits[j], ylim=limits[i + 1])
#         kws = dict(color='black', alpha=0.1, lw=0.5, zorder=0)
#         axes[i, j].axvline(0.0, **kws)
#         axes[i, j].axhline(0.0, **kws)
# # plt.close()

# def update(turn):
#     turn *= 10
#     X = coords_foil_frame[turn]
#     for i in range(3):
#         for j in range(i + 1):
#             lines[i][j].set_data(X[:, j], X[:, i + 1])
#     for text in axes[0, 1].texts:
#         text.set_visible(False)
#     axes[0, 1].annotate('Turn = {}'.format(turn), xy=(0.4, 0.5), xycoords='axes fraction')
    
# #     for ax in axes:
# #         for patch in ax.patches:
# #             patch.set_visible(False)
#     Sigma = np.cov(X.T)
#     centers = np.mean(X, axis=0)
#     myplt.rms_ellipses(4 * Sigma, centers=centers, axes=axes, color='black', zorder=99)

# anim = animation.FuncAnimation(fig, update, frames=30)
# anim.save('_output/figures/cornertest.mp4', dpi=350)

In [ ]:
anim = myanim.corner(coords_foil_frame[:500], dims=4, **anim_kws)
anim.save('_output/figures/corner4D.mp4', dpi=350, fps=4)

In [ ]:
anim = myanim.corner(coords_foil_frame, dims=6, **anim_kws)
anim.save('_output/figures/corner6D.mp4', dpi=350, fps=5)

In [ ]:
i = 0
X_onepart = np.array([X[i, :] for X in coords_foil_frame])

axes = myplt.corner(X_onepart, kind='scatter', c='steelblue', pad=0.1)
plt.savefig('_output/figures/corner_part{}.png'.format(i), **savefig_kws)

In [ ]:
# anim = myanim.corner_onepart(
#     X_onepart[:50], show_history=True, skip=0, pad=0.35, text_fmt='Turn = {}', 
#     zero_center=False, history_kws=dict(ms=5, color='lightgrey'),
# )
# anim.save('_output/figures/corner_part{}.mp4'.format(i), dpi=350, fps=5)